In [2]:
from model import Actor, Critic, DRRAveStateRepresentation, PMF
from learn import DRRTrainer
from utils.general import csv_plot
import torch
import pickle
import numpy as np
import random
import os
import datetime

import matplotlib.pyplot as plt
 
%matplotlib inline

In [3]:
class config():
    output_path = 'results/' + datetime.datetime.now().strftime('%y%m%d') + '/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    plot_dir = output_path + 'rewards.pdf'
 
    train_actor_loss_data_dir = output_path + 'train_actor_loss_data.npy'
    train_critic_loss_data_dir = output_path + 'train_critic_loss_data.npy'
    train_mean_reward_data_dir = output_path + 'train_mean_reward_data.npy'
 
    train_actor_loss_plot_dir = output_path + 'train_actor_loss.png'
    train_critic_loss_plot_dir = output_path + 'train_critic_loss.png'
    train_mean_reward_plot_dir = output_path + 'train_mean_reward.png'
 
    trained_models_dir = 'trained/'
 
    actor_model_trained = trained_models_dir + 'actor_net.weights'
    critic_model_trained = trained_models_dir + 'critic_net.weights'
    state_rep_model_trained = trained_models_dir + 'state_rep_net.weights'
 
    actor_model_dir = output_path + 'actor_net.weights'
    critic_model_dir = output_path + 'critic_net.weights'
    state_rep_model_dir = output_path + 'state_rep_net.weights'
 
    csv_dir = output_path + 'log.csv'
 
    path_to_trained_pmf = trained_models_dir + 'trained_pmf.pt'
 
    # hyperparams
    batch_size = 64
    gamma = 0.9
    replay_buffer_size = 100000
    history_buffer_size = 5
    learning_start = 5000
    learning_freq = 1
    lr_state_rep = 0.001
    lr_actor = 0.0001
    lr_critic = 0.001
    eps_start = 1
    eps = 0.1
    eps_steps = 10000
    eps_eval = 0.1
    tau = 0.01 # inital 0.001
    beta = 0.4
    prob_alpha = 0.3
    max_timesteps_train = 260000
    max_epochs_offline = 500
    max_timesteps_online = 20000
    embedding_feature_size = 100
    episode_length = 10
    train_ratio = 0.8
    weight_decay = 0.01
    clip_val = 1.0
    log_freq = 100
    saving_freq = 1000
    zero_reward = False
 
    enable_cuda = False
 
def seed_all(cuda, seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.empty_cache()
        torch.cuda.manual_seed(seed=seed)
 
print("Initializing DRR Framework ----------------------------------------------------------------------------")
 
# Get CUDA device if available
cuda = True if config.enable_cuda and torch.cuda.is_available() else False
print("Using CUDA") if cuda else print("Using CPU")
 
# Init seeds
seed_all(cuda, 0)
print("Seeds initialized")
 
# Grab models
actor_function = Actor
critic_function = Critic
state_rep_function = DRRAveStateRepresentation
 
# Import Data
users = pickle.load(open('dataset/user_id_to_num.pkl', 'rb'))
items = pickle.load(open('dataset/rest_id_to_num.pkl', 'rb'))
data = np.load('dataset/data.npy')

# Normalize rewards to [-1, 1]
data[:, 0] = data[:, 0].astype(float)
print(data.dtype)
data[:, 0] = 0.5 * (data[:, 0] - 3)
print(data)
 
np.random.shuffle(data)
train_data = torch.from_numpy(data[:int(config.train_ratio * data.shape[0])])
test_data = torch.from_numpy(data[int(config.train_ratio * data.shape[0]):])
print("Data imported, shuffled, and split into Train/Test, ratio=", config.train_ratio)
print("Debug: train_data", train_data)
print("Train data shape: ", train_data.shape)
print("Test data shape: ", test_data.shape)
 
# Create and load PMF function for rewards and embeddings
n_users = len(users)
n_items = len(items)
reward_function = PMF(n_users, n_items, config.embedding_feature_size, is_sparse=False, enable_cuda=cuda)
reward_function.load_state_dict(torch.load(config.path_to_trained_pmf))
 
# Freeze all the parameters in the network
for param in reward_function.parameters():
    param.requires_grad = False
print("Initialized PMF, imported weights, created reward_function")
 
# Extract embeddings
user_embeddings = reward_function.user_embeddings.weight.data
item_embeddings = reward_function.item_embeddings.weight.data
print("Extracted user and item embeddings from PMF")
print("User embeddings shape: ", user_embeddings.shape)
print("Item embeddings shape: ", item_embeddings.shape)
 
# Init trainer
print("Initializing DRRTrainer -------------------------------------------------------------------------------")
trainer = DRRTrainer(config,
                      actor_function,
                      critic_function,
                      state_rep_function,
                      reward_function,
                      users,
                      items,
                      train_data,
                      test_data,
                      user_embeddings,
                      item_embeddings,
                      cuda
                      )

Initializing DRR Framework ----------------------------------------------------------------------------
Using CPU
Seeds initialized
int64
[[         0 1531001351          0 ...    1575913         33      80739]
 [         1 1325604498          0 ...     808602         10      19425]
 [         0 1391632230          0 ...     301895       1332      83879]
 ...
 [         0 1577678190          0 ...    1578206        326      65728]
 [         1 1642618767          0 ...    1117070        281      67627]
 [         0 1514933447          3 ...     227711       1868      75424]]
Data imported, shuffled, and split into Train/Test, ratio= 0.8
Debug: train_data tensor([[         1, 1546693532,          0,  ...,    1404279,          7,
             118750],
        [         0, 1396451829,          0,  ...,     690079,          5,
              30670],
        [         1, 1494989962,          0,  ...,     316522,         23,
             130056],
        ...,
        [         1, 1625455155, 

/tmp/ipykernel_4961/913719725.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  reward_function.load_state_dict(torch.load(config.path_to_trained_pmf))


Initialized PMF, imported weights, created reward_function
Extracted user and item embeddings from PMF
User embeddings shape:  torch.Size([1987843, 100])
Item embeddings shape:  torch.Size([150346, 100])
Initializing DRRTrainer -------------------------------------------------------------------------------
CUDA Device ID:  0
NVIDIA GeForce RTX 3050 Ti Laptop GPU
CUDA Memory Allocated:  0
CUDA Memory Reserved:  0.0 GB
Current PyTorch Device:  cpu
Data dimensions extracted
Models initialized
Model weights initialized, copied to target
Optimizers initialized


In [5]:
# Train
print("Starting DRRTrainer.learn() ---------------------------------------------------------------------------")
actor_losses, critic_losses, epi_avg_rewards = trainer.learn()

Starting DRRTrainer.learn() ---------------------------------------------------------------------------
Debug: start learning
tensor([[5.0000e-01, 1.3127e+09, 2.0000e+00, 1.0000e+00, 4.0000e+00, 1.5934e+06,
         1.5000e+01, 8.0878e+04],
        [5.0000e-01, 1.3200e+09, 0.0000e+00, 1.0000e+00, 1.0000e+00, 1.5934e+06,
         1.5000e+01, 6.0695e+04],
        [5.0000e-01, 1.5010e+09, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.5934e+06,
         1.5000e+01, 1.7142e+04],
        [1.0000e+00, 1.5287e+09, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5934e+06,
         1.5000e+01, 4.8691e+04],
        [1.0000e+00, 1.5322e+09, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.5934e+06,
         1.5000e+01, 6.3109e+04],
        [1.0000e+00, 1.5788e+09, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5934e+06,
         1.5000e+01, 2.6781e+04]], dtype=torch.float64)


IndexError: tensors used as indices must be long, int, byte or bool tensors

In [18]:
# Change to newest trained data directories
config.trained_models_dir = config.output_path
output_path = config.output_path
# config.trained_models_dir = "results/210419-181221/"
# output_path = "results/210419-181221/"

train_actor_loss_data_dir = output_path + 'train_actor_loss_data.npy'
train_critic_loss_data_dir = output_path + 'train_critic_loss_data.npy'
train_mean_reward_data_dir = output_path + 'train_mean_reward_data.npy'

config.actor_model_trained = config.trained_models_dir + 'actor_net.weights'
config.critic_model_trained = config.trained_models_dir + 'critic_net.weights'
config.state_rep_model_trained = config.trained_models_dir + 'state_rep_net.weights'

In [32]:
!pip install tsmoothie

def noiseless_plot(y, title, ylabel, save_loc):
  # operate smoothing
  smoother = ConvolutionSmoother(window_len=1000, window_type='ones')
  smoother.smooth(y)

  # generate intervals
  low, up = smoother.get_intervals('sigma_interval', n_sigma=3)

  # plot the smoothed timeseries with intervals
  plt.close()
  plt.figure(figsize=(11,6))
  plt.xlabel("Epoch") 
  plt.ylabel(ylabel)
  plt.title(title)
  plt.plot(smoother.data[0], color='orange')
  plt.plot(smoother.smooth_data[0], linewidth=3, color='blue')
  plt.fill_between(range(len(smoother.data[0])), low[0], up[0], alpha=0.3)
  plt.savefig(save_loc)
  plt.close()


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [29]:
import matplotlib.pyplot as plt
from tsmoothie.smoother import *

actor_losses = np.load(train_actor_loss_data_dir)
critic_losses = np.load(train_critic_loss_data_dir)
epi_avg_rewards = np.load(train_mean_reward_data_dir)

noiseless_plot(actor_losses, 
               "Actor Loss (Train)", 
               "Actor Loss (Train)", 
               output_path + "train_actor_loss_smooth.png")
               
noiseless_plot(critic_losses, 
               "Critic Loss (Train)", 
               "Critic Loss (Train)", 
               output_path + "train_critic_loss_smooth.png")

noiseless_plot(epi_avg_rewards, 
               "Mean Reward (Train)", 
               "Mean Reward (Train)", 
               output_path + "train_mean_reward_smooth.png")

NameError: name 'smoother' is not defined

Save hyperparameters.

In [12]:
sourceFile = open(output_path + "hyperparams.txt", 'w')
print(config.__dict__, file = sourceFile)
sourceFile.close()

**Run** Offline and Online evaluations, save scores.

In [11]:
T_precisions = [5, 10, 15, 20]

In [13]:
# Offline evaluate

# PMF
for T_precision in T_precisions:
  pmf_Ts = []
  for i in range(20):
    # Evaluate
    avg_precision = trainer.offline_pmf_evaluate(T_precision)

    # Append to list
    pmf_Ts.append(avg_precision)

  # Save data
  pmf_Ts = np.array(pmf_Ts)
  np.save(output_path + f'avg_precision@{T_precision}_offline_pmf_eval.npy', pmf_Ts)

  # Save
  sourceFile = open(output_path + f'avg_precision@{T_precision}_offline_pmf_eval.txt', 'w')
  print(f'Average Precision@{T_precision} (Eval): {np.mean(pmf_Ts)}', file=sourceFile)
  sourceFile.close()

# DRR
# for T_precision in T_precisions:
#   drr_Ts = []
#   for i in range(20):
#     # Evaluate
#     avg_precision = trainer.offline_evaluate(T_precision)

#     # Append to list
#     drr_Ts.append(avg_precision)

#   # Save data
#   drr_Ts = np.array(drr_Ts)
#   np.save(output_path + f'avg_precision@{T_precision}_offline_eval.npy', drr_Ts)

#   # Save
#   sourceFile = open(output_path + f'avg_precision@{T_precision}_offline_eval.txt', 'w')
#   print(f'Average Precision@{T_precision} (Eval): {np.mean(drr_Ts)}', file=sourceFile)
#   sourceFile.close()

/home/tori/Tori/DRL-based-Recommendation/learn.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.actor_net.load_state_dict(torch.load(self.config.actor_model_traine

FileNotFoundError: [Errno 2] No such file or directory: 'results/241012-14/state_rep_net.weights'

In [14]:
pmf_fives = np.load(output_path + 'avg_precision@5_offline_pmf_eval.npy')
pmf_tens = np.load(output_path + 'avg_precision@10_offline_pmf_eval.npy')
pmf_fifteens = np.load(output_path + 'avg_precision@15_offline_pmf_eval.npy')
pmf_twenties = np.load(output_path + 'avg_precision@20_offline_pmf_eval.npy')

drr_fives = np.load(output_path + 'avg_precision@5_offline_eval.npy')
drr_tens = np.load(output_path + 'avg_precision@10_offline_eval.npy')
drr_fifteens = np.load(output_path + 'avg_precision@15_offline_eval.npy')
drr_twenties = np.load(output_path + 'avg_precision@20_offline_eval.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'results/241012-14/avg_precision@5_offline_pmf_eval.npy'

In [15]:
Ts = [5, 10, 15, 20]

In [ ]:
# Online evaluate
for T in Ts:
  avgs = []
  # Change T
  config.episode_length = T
  for i in range(20):
    # Evaluate
    avg_reward = trainer.online_evaluate()

    # Append data
    avgs.append(avg_reward)

  # Save data
  avgs = np.array(avgs)
  np.save(output_path + f'avg_reward@{T}_online_eval.npy', avgs)

  # Save
  sourceFile = open(output_path + f'avg_reward@{T}_online_eval.txt', 'w')
  print(f'Average Reward@{T} (Eval): {np.mean(avgs)}', file=sourceFile)
  sourceFile.close()

In [11]:
online_fives = np.load(output_path + 'avg_reward@5_online_eval.npy')
online_tens = np.load(output_path + 'avg_reward@10_online_eval.npy')
online_fifteens = np.load(output_path + 'avg_reward@15_online_eval.npy')
online_twenties = np.load(output_path + 'avg_reward@20_online_eval.npy')

In [16]:
    # Evaluation @K Graphing

    def createEvalPlot(title, ylabel, xlabel, filename, x, y, e, e_x_off, e_y_off):
      plt.figure()
      plt.errorbar(x, y, yerr=e, fmt='.-', ecolor="red", capsize=3)
      plt.title(title)
      plt.xlabel(xlabel)
      plt.ylabel(ylabel)
      plt.xticks(x)
      for i, j in zip(x, y):
          plt.annotate(str(round(j, 4)), xy=(i+e_x_off, j+e_y_off))
      plt.savefig(filename)

    # Combine data
    pmf_offline_eval_data = [pmf_fives, pmf_tens, pmf_fifteens, pmf_twenties]
    offline_eval_data = [drr_fives, drr_tens, drr_fifteens, drr_twenties]
    online_eval_data = [online_fives, online_tens, online_fifteens, online_twenties]

    # Calculate means and stds for graphing
    pmf_offline_means, pmf_offline_stds = [], []
    offline_means, offline_stds = [], []
    online_means, online_stds = [], []
    for d in pmf_offline_eval_data:
        pmf_offline_means.append(np.mean(d))
        pmf_offline_stds.append(np.std(d))

    for d in offline_eval_data:
        offline_means.append(np.mean(d))
        offline_stds.append(np.std(d))

    for d in online_eval_data:
        online_means.append(np.mean(d))
        online_stds.append(np.std(d))

    print(pmf_offline_means)
    print(pmf_offline_stds)
    print(offline_means)
    print(offline_stds)
    print(online_means)
    print(online_stds)


    # Create and save eval plots
    createEvalPlot("Average Precision @K for Offline PMF Evaluation\n(500 random users, K recommendations each, 20 times)",
                   "Average Precision @K",
                   "K",
                   output_path + "pmf_offline_eval.png",
                   T_precisions,
                   pmf_offline_means,
                   pmf_offline_stds,
                   0.4,
                   0)

    createEvalPlot("Average Precision @K for Offline DRR Evaluation\n(500 random users, K recommendations each, 20 times)",
                   "Average Precision @K",
                   "K",
                   output_path + "offline_eval.png",
                   T_precisions,
                   offline_means,
                   offline_stds,
                   0.4,
                   0)

    createEvalPlot(
        "Average Reward @K for Online DRR Evaluation\n(20,000 recommendations at each K, 20 times)",
        "Average Reward @K",
        "K",
        output_path + "online_eval.png",
        Ts,
        online_means,
        online_stds,
        0.3,
        -0.004)

NameError: name 'pmf_fives' is not defined